In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import chardet
import os

In [4]:
set_diff_df=[]
val=[]
name_list=pd.read_excel('/content/drive/My Drive/Attendance/Name_list.xlsx')
name_list['Roll Number']=name_list['Roll Number'].str.strip()
reg_no_full=name_list['Roll Number']
directory = os.path.join("/content/drive/My Drive/","Attendance")
for root,dirs,files in os.walk(directory):
    for file in files:
       if file.endswith(".csv"):
           path='/content/drive/My Drive/Attendance/'
           with open(path+file, 'rb') as f:
              result = chardet.detect(f.read()) 
           data = pd.read_csv(path+file,encoding=result['encoding'],sep='\t') 
           data.sort_values("Full Name", inplace = True) 
           data.drop_duplicates(subset ="Full Name",keep ='first', inplace = True)
           data['Reg No']=data['Full Name'].apply(lambda st: st[st.find("[")+1:st.find("]")])
           data["Timestamp"] = data["Timestamp"].apply(pd.to_datetime)
           present=data['Reg No']
           set_diff_df.append((pd.concat([reg_no_full, present, present]).drop_duplicates(keep=False)).to_string(index=False))
           val.append(str(data['Timestamp'][0].date().day)+'-'+str(data['Timestamp'][0].date().month)+'-'+str(data['Timestamp'][0].date().year))
           
new_val= pd.DataFrame(list(zip(val, set_diff_df)),columns =['date', 'absentees']) 
new_val.to_excel('/content/drive/My Drive/Attendance/consolidated.xlsx',index=False)